<div class="alert alert-block alert-success">
<b>Acknowledgements:</b> This hands-on session is based on a notebook from <a href="https://www.drcmr.dk/marcop">Dr Marco Pizzolato</a> of the Danish Reseaerch Center for Magnetic Resonance (Denmark).
</div>

# Magnetic resonance imaging and Fourier Transform

The raw data, i.e. the *k-space* that was acquired by the MRI scanner, is contained in the file `oneslice.nii`.

**NB:** For simplicity, in this exercise we'll work with data consisting only of a **single slice**, but all the results may be extended to any dimension.

## Setup the experiments

Let's first install some *useful libraries* from `https://pypi.org` using `pip`:

In [ ]:
%pip install numpy
%pip install matplotlib
%pip install nibabel

Then, let's define some *useful variables and functions*:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import nibabel
%matplotlib inline

# define some useful variables
FONT_SIZE         = 20
FIG_SIZE          = [18,9]
ARROW_HEAD_LENGTH = 5.
ARROW_HEAD_WIDTH  = 3.

def calculate_sinusoid( dimension, wx, wy, fx, fy, phix=0.0, phiy=0.0 ):
    '''Returns a 2D image (of size "dimension") with a sinusoidal phase between -pi and +pi.

    dimension: tuple specifying size (size x,size y)
    wx       : x period length
    wy       : y period length
    fx       : x frequency
    phix     : x initial phase
    fy       : y frequency
    phiy     : y initial phase
    '''
    if len(dimension) != 2:
        raise ValueError('dimension must be a tuple of length 2')

    sin_xy = np.zeros(dimension)
    for y in range(sin_xy.shape[0]):
        for x in range(sin_xy.shape[1]):
            sin_xy[x,y] = np.pi * np.sin(2*np.pi*fx*x/wx+phix+2*np.pi*fy*y/wy+phiy)
    return sin_xy

def plot_arrows( Cx, Cy, Nx, Ny, color='r' ):
    '''Display the x and y axes of the image as arrows'''
    plt.arrow( Cx, 0, 0, Ny-1, length_includes_head=True, head_width=ARROW_HEAD_WIDTH, head_length=ARROW_HEAD_LENGTH, fc=color, ec=color, lw=2, ls='-', alpha=1.0 );
    plt.arrow( 0, Cy, Nx-1, 0, length_includes_head=True, head_width=ARROW_HEAD_WIDTH, head_length=ARROW_HEAD_LENGTH, fc=color, ec=color, lw=2, ls='-', alpha=1.0 );

## Open the raw data

Let's **open the raw data** and print some information about it:

In [ ]:
# open the file "openslice.nii"
nii_oneslice = nibabel.load( 'oneslice.nii' )
kspace = np.asarray( nii_oneslice.dataobj )

# size of k-space
Nx = kspace.shape[0]
Ny = kspace.shape[1]

# center of k-space
Cx = Nx/2.0
Cy = Ny/2.0

print( f'dimension      = {Nx}x{Ny}' )
print( f'datatype       = {kspace.dtype}' )
print( f'k-space center = [{Cx:.1f}, {Cy:.1f}]' )

The acquisition Field Of View (FOV) was 220 *mm* (not stored in the header), so the **spatial resolution** (size of each pixel) is:

In [ ]:
# resolution = FOV / # voxels
resolution = 220.0 / Nx
print( f'The spatial resolution is {resolution:.2f} mm' )

# frequency = 1/resolution
K_max = 1. / resolution
print( f'which corresponds to a maximum frequency of {K_max:.2f} mm^-1' )

## Visualize the k-space

Remember that the k-space is a **complex image**, so we cannot visualize it "as is", but we have to plot separately its *real* and *imaginary* parts:

In [ ]:
plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )

# real part
plt.subplot( 121 )
max_val = np.abs(kspace.real).max()
plt.imshow(
    kspace.real, origin='lower',
    cmap='seismic', clim=[-max_val, max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'r' )
plt.xlabel( '$K_x$', fontsize=FONT_SIZE )
plt.ylabel( '$K_y$', fontsize=FONT_SIZE )
plt.title( 'real part', fontsize=FONT_SIZE )

# imaginary part
plt.subplot( 122 )
max_val = np.abs(kspace.imag).max()
plt.imshow(
    kspace.imag, origin='lower',
    cmap='seismic', clim=[-max_val, max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'r' )
plt.xlabel ('$K_x$', fontsize=FONT_SIZE )
plt.ylabel ('$K_y$', fontsize=FONT_SIZE )
plt.title( 'imaginary part', fontsize=FONT_SIZE );

## Visualization as "magnitude" and "phase"

The k-space is a complex image K:

\begin{equation}
K(x,y) = \Re[{K(x,y)}] + i\cdot \Im[{K(x,y)}]
\end{equation}

It's **magnitude** is:

\begin{equation}
|K(x,y)| = \sqrt{\Re[{K(x,y)}]^2 +  \Im[{K(x,y)}]^2}
\end{equation}

and it's **phase** is:

\begin{equation}
\Phi(x,y) = arctan  \left( \frac{\Im[K(x,y)]}{\Re[K(x,y)]} \right)
\end{equation}

So, let's **plot separately** the *magnitude* and *phase* of the k-space:

In [ ]:
plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )

# magnitude image
plt.subplot( 121 )
max_val = np.abs(kspace).max() / 10.0 # this division is to improve the contrast
plt.imshow(
    np.abs(kspace), origin='lower',
    cmap='viridis', clim=[0,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'r')
plt.xlabel( '$K_x$', fontsize=FONT_SIZE )
plt.ylabel( '$K_y$', fontsize=FONT_SIZE )
plt.title( 'magnitude', fontsize=FONT_SIZE )

# phase image
plt.subplot( 122 )
plt.imshow(
    np.angle(kspace), origin='lower',
    cmap='seismic', clim=[-np.pi,np.pi]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'k' )
plt.xlabel( '$K_x$', fontsize=FONT_SIZE )
plt.ylabel( '$K_y$', fontsize=FONT_SIZE )
plt.title(  'phase', fontsize=FONT_SIZE );

# Reconstruct the image in *spatial coordinates*

To visualize the actual brain image in **spatial coordinates**, we need to apply the 2D Fourier transform to the raw data and to pass into the so-called "image space".

## Change the convention for the Fast Fourier Transform (FFT)

To achieve this in Python, we first need to *shift the zero frequency to the corners of the image* using the function [`np.fft.fftshift`](https://numpy.org/doc/stable/reference/generated/numpy.fft.fftshift.html); this step is required because of the actual implementation of the Fast Fourier Transform:

In [ ]:
kspace_shifted = np.fft.fftshift( kspace )

# visualize the shifted k-space
plt.figure( figsize=FIG_SIZE )
plt.imshow(
    np.abs(kspace_shifted), origin='lower',
    cmap='viridis', clim=[0,max_val]
)
plt.colorbar( shrink=0.75 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( 0,0,Nx,Ny,'r' )
plt.title( 'magnitude (shifted)', fontsize=FONT_SIZE );

Now, we can apply the 2D Fourier Transform to the shifted k-space to obtain the image in *spatial coordinates*:

In [ ]:
img = np.fft.fft2( kspace_shifted / np.sqrt(Nx*Ny), norm='ortho' )

NB: note that we had to *scale the values* by the square root of the number of samples, again because of implementation choices of the Fast Fourier Transform of `numpy`.

## Visualize the reconstructed image as *real* and *imaginary* parts

Remember, it is again a complex image:

In [ ]:
plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )

# real part
plt.subplot( 121 )
max_val = np.abs(img.real).max()
plt.imshow(
    img.real, origin='lower',
    cmap='seismic', clim=[-max_val,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'k' )
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )
plt.title( 'Reconstructed image (real)', fontsize=FONT_SIZE )

# imaginary part
plt.subplot( 122 )
max_val = np.abs(img.imag).max()
plt.imshow(
    img.imag, origin='lower',
    cmap='seismic', clim=[-max_val,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'k' )
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE );
plt.title(  'Reconstructed image (imaginary)', fontsize=FONT_SIZE );

## Visualization as *magnitude* and *phase*

In [ ]:
plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )

# magnitude
plt.subplot( 121 )
max_val = np.abs(img).max() / 3.0 # this division is to improve contrast
plt.imshow(
    np.abs(img), origin='lower',
    cmap='gray', clim=[0,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'r' )
plt.title( 'Reconstructed image (magnitude)', fontsize=FONT_SIZE )
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )

# phase
plt.subplot( 122 )
plt.imshow(
    np.angle(img), origin='lower',
    cmap='seismic', clim=[-np.pi,np.pi]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'k' )
plt.title( 'Reconstructed image (phase)', fontsize=FONT_SIZE )
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE );

The *pixel intensity* in the magnitude image is the magnitude of the magnetization vector for that location.

# Inspect and reconstruct from a portion of the k-space

We will now select a rectangular **portion of the k-space** corresponding to a given range of frequencies:

In [ ]:
# Offset of the the selected rectangle (w.r.t. the center of k-space)
OFFSETx = 0
OFFSETy = 0

# Side of the selected rectangle
Kx_half_side = 3
Ky_half_side = 3

# The corresponding pixels coordinates are
x_start = int(Cx) + OFFSETx - Kx_half_side
x_stop  = int(Cx) + OFFSETx + Kx_half_side
y_start = int(Cy) + OFFSETy - Ky_half_side
y_stop  = int(Cy) + OFFSETy + Ky_half_side
print( f'Selected portion of k-space data (w.r.t. center) = [{x_start-int(Cx)},{x_stop-int(Cx)},{y_start-int(Cy)},{y_stop-int(Cy)}]' )

kspace_mod = kspace[y_start:y_stop+1,x_start:x_stop+1]

# Mask to define the selected frequencies
frequency_mask = np.zeros( (Nx,Ny), dtype=bool )
frequency_mask[y_start:y_stop+1,x_start:x_stop+1] = True

## Visualize the selected portion

In [ ]:
fig = plt.figure( figsize=FIG_SIZE )
max_val = np.abs(kspace).max() / 5.0 # this division is to improve the contrast
plt.imshow(
    np.abs(kspace), origin='lower',
    cmap='viridis', clim=[0,max_val]
)
plt.colorbar( shrink=0.75 )

# box
rect = patches.Rectangle(
    [x_start-0.5, y_start-0.5],
    2*Kx_half_side+1, 2*Ky_half_side+1,
    linewidth=2, edgecolor='w', facecolor='none', zorder=200
)
plt.gca().add_patch( rect )

plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plot_arrows( Cx,Cy,Nx,Ny,'r' )
plt.xlabel( '$K_x$', fontsize=FONT_SIZE )
plt.ylabel( '$K_y$', fontsize=FONT_SIZE )
plt.title(  'Selected area of the k-space', fontsize=FONT_SIZE );

## Plot the sinusoids corresponding to the selected area of the k-space

Display the **2D sinusoids** corresponding to the low-frequency portion we just selected with the `frequency_mask` mask (maximum 40 for sake of space):

In [ ]:
plt.figure( figsize=FIG_SIZE )
image_number = 0
for iy in range(Ny)[::-1]:  # display in reverse order to match previous plot
    for ix in range(Nx):
        if frequency_mask[iy,ix] == True:
            image_number += 1
            if image_number>40:
                break

            K_x = K_max/2. * (ix-Cx)/Cx
            K_y = K_max/2. * (iy-Cy)/Cy
            sin2D = calculate_sinusoid( (Ny,Nx), K_max,K_max, K_y,K_x )

            plt.subplot( 4, 10, image_number )
            plt.imshow( sin2D, origin='lower' )
            plt.tick_params(
                reset=True, axis='both', which='both',
                left=False, right=False, bottom=False, top=False,
                labelbottom=False, labelleft=False
            )
            plt.text(
                Cx, Cy, f'({ix-int(Cx)},{iy-int(Cy)})',
                color='k', backgroundcolor='w', fontsize=10, ha='center'
            );

## Try setting to 0 all frequencies outside the selected region

Modify the k-space by setting all frequencies outside the selected region to 0:

In [ ]:
# modified k-space
kspace_mod = kspace.copy()
kspace_mod[frequency_mask==0] = 0. + 1j*0.

# visualize
fig = plt.figure( figsize=FIG_SIZE )
max_val = np.abs(kspace).max() / 5.0 # this division is to improve the contrast
plt.imshow(
    np.abs(kspace_mod), origin='lower',
    cmap='viridis', clim=[0,max_val]
)
plt.colorbar( shrink=0.75 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( '$K_x$', fontsize=FONT_SIZE )
plt.ylabel( '$K_y$', fontsize=FONT_SIZE )
plt.title(  'Selected area of the k-space', fontsize=FONT_SIZE );

## Regenerate the images

Now, let's reconstruct the brain image (in spatial coordinates) **only** from the selected portion of frequencies we just selected, i.e. `kspace_less_frequencies`:

In [ ]:
# reconstruct from frequencies in the selected portion
kspace_mod_shifted = np.fft.fftshift( kspace_mod )
img_mod = np.fft.fft2( kspace_mod_shifted / np.sqrt(Nx*Ny), norm='ortho' )

# visualize the original image
plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )
plt.subplot( 121 )
max_val = np.max(np.abs(img)) / 3.0 # this division is to improve contrast
plt.imshow(
    np.abs(img), origin='lower',
    cmap='gray', clim=[0,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )
plt.title(  'From ALL FREQs (magnitude)', fontsize=FONT_SIZE )

# visualize the image from fewer frequencies
plt.subplot( 122 )
max_val = np.max(np.abs(img_mod))
plt.imshow(
    np.abs(img_mod), origin='lower',
    cmap='gray', clim=[0,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )
plt.title(  'From PORTION (magnitude)', fontsize=FONT_SIZE );

## Exercises

Try changing the portion of frequencies to be considered, in particular:

- select a larger area by increasing `Kx_half_side` and `Ky_half_side`
- move the box by changing `OFFSETx` and `OFFSETy`
- set to zero only the low frequencies
- ...

and see the effects.

# Simulate an interference

We keep the whole k-space, but we set a point of the k-space to a MUCH MUCH bigger value...

In [ ]:
# offset w.r.t. the center of k-space
OFFSETx = -25
OFFSETy = 25

# amplitude of the interference
amplification_factor = 1e5

# modified k-space
kspace_mod = kspace.copy()
kspace_mod[int(Cy)+OFFSETy,int(Cx)+OFFSETx] = amplification_factor * (1. + 1j*1)

## Visualize the frequency we are amplifying

In [ ]:
# compute the 2D sinusoid
K_x = K_max/2. *(OFFSETx-Cx)/Cx
K_y = K_max/2. *(OFFSETy-Cy)/Cy
sin2D = calculate_sinusoid( (Ny,Nx), K_max,K_max, K_y,K_x )

plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )

# visualize the position in k-space
plt.subplot( 121 )
max_val = np.abs(kspace).max() / 5.0 # this division is to improve the contrast
plt.imshow(
    np.abs(kspace), origin='lower',
    cmap='viridis', clim=[0,max_val]
)
plt.colorbar( shrink=0.75 )
plot_arrows( Cx,Cy,Nx,Ny,'r' )
plt.plot( Cx+OFFSETx, Cy+OFFSETy, 'wo', markersize=5 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( '$K_x$', fontsize=FONT_SIZE )
plt.ylabel( '$K_y$', fontsize=FONT_SIZE )
plt.title( 'Selected area of the k-space', fontsize=FONT_SIZE );

# plot the corresponding 2D sinusoid
plt.subplot( 122 )
plt.imshow( sin2D, origin='lower' )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.text(
    Cx, Cy, f'({OFFSETx},{OFFSETy})',
    color='k', backgroundcolor='w', fontsize=FONT_SIZE, ha='center'
)
plt.title( 'Corresponding sinusoid', fontsize=FONT_SIZE );

## Visualize the effect on the image

In [ ]:
# Reconstruct the image from the modified k-space
kspace_mod_shifted = np.fft.fftshift( kspace_mod )
img_mod = np.fft.fft2( kspace_mod_shifted / np.sqrt(Nx*Ny), norm='ortho' )

# Display and compare
plt.figure( figsize=FIG_SIZE )
plt.subplots_adjust( wspace=0.4 )

plt.subplot(121)
max_val = np.max(np.abs(img)) / 3.0 # this division is to improve contrast
plt.imshow(
    np.abs(img), origin='lower',
    cmap='gray', clim=[0,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )
plt.title(  'Original image (magnitude)', fontsize=FONT_SIZE )

plt.subplot(122)
max_val = np.max(np.abs(img_mod)) # uncommment to display within its own intensity range
plt.imshow(
    np.abs(img_mod), origin='lower',
    cmap='gray', clim=[0,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )
plt.title(  'WITH interference (magnitude)', fontsize=FONT_SIZE );

## Visualize the difference between the 2 images

In [ ]:
# Compute the actual difference
difference = np.abs(img) - np.abs(img_mod)

plt.figure( figsize=FIG_SIZE )
max_val = np.abs(difference).max()
plt.imshow(
    difference, origin='lower',
    cmap='coolwarm', clim=[-max_val,max_val]
)
plt.colorbar( shrink=0.65 )
plt.tick_params(
    reset=True, axis='both', which='both',
    left=False, right=False, bottom=False, top=False,
    labelbottom=False, labelleft=False
)
plt.xlabel( 'x pixels', fontsize=FONT_SIZE )
plt.ylabel( 'y pixels', fontsize=FONT_SIZE )
plt.title(  'Difference', fontsize=FONT_SIZE );

## Exercise

Try changing some parameters, e.g. `OFFSETx`, `OFFSETy` and `amplification_factor`, and see the effects.